In [ ]:
import cv2
from PIL import Image, ImageDraw, ImageFont
import random
import os
import json
import numpy as np
from IPython.display import Image as ImageDisplay, display
import string
from faker import Faker
fake = Faker('en_US')
def label_to_json(label_data, image_path, json_filepath, height, width):
    json_data = {
        "path": image_path,
        "outputs": {
            "object": []
        },
        "time_labeled": 1667818663351,
        "labeled": True,
        "size": {
            "width": height,
            "height": width,
            "depth": 3
        }
    }

    for parts in label_data:
        label = parts[0]
        xmin = float(parts[1])
        ymin = float(parts[2])
        xmax = float(parts[3])
        ymax = float(parts[4])
        
        json_data["outputs"]["object"].append({
            "name": str(label),
            "bndbox": {
                "xmin": xmin,
                "ymin": ymin,
                "xmax": xmax,
                "ymax": ymax
            }
        })

        with open(json_filepath, "w") as json_file:
            json.dump(json_data, json_file, indent=4)

    return json_data


def random_size(output_path, boxes):
    img = cv2.imread(output_path)

    width, height = img.shape[1], img.shape[0]
    new_img_width = random.randint(400, 1200)
    resize_factor = new_img_width/width
    new_img_height = int(height*(resize_factor))
    
    img = cv2.resize(img, (new_img_width, new_img_height))
    
    zoom = random.uniform(1,1.5)
    background = (55,100,10)
    
    new_h, new_w = int(new_img_height*zoom), int(new_img_width*zoom)
    
    ## bg_img = Image.new('RGB', (new_w, new_h), background)
    # bcfolder = "background/"
    # bc_list = os.listdir(bcfolder)
    # bg_img = Image.open('background/' + bc_list[random.randint(0, len(bc_list)-1)])
    # bg_img = bg_img.resize((new_w, new_h))
    
    # offset_w = abs(int((new_w-new_img_width)/2))
    # offset_h = abs(int((new_h-new_img_height)/2))
    
    # offset_w = offset_w + random.randint(-offset_w, offset_w)
    # offset_h = offset_h + random.randint(-offset_h, offset_h)
    
    # bg_img.paste(Image.fromarray(img), (offset_w, offset_h))
    
    # new_img = cv2.cvtColor(np.array(bg_img), cv2.COLOR_BGR2RGB)
    
    new_img = img
    offset_h, offset_w = 0,0
    
    #Boxes correction
    corrected_box = []
    for box in boxes:
        xmin = int(box[1]*resize_factor)+offset_w
        ymin = int(box[2]*resize_factor)+offset_h
        xmax = int(box[3]*resize_factor)+offset_w
        ymax = int(box[4]*resize_factor)+offset_h
        corrected_box.append(tuple([box[0], xmin, ymin, xmax, ymax]))
    
    return new_img, corrected_box

def perspective_warp(image, boxes, max_distortion=0.2):
    height, width = image.shape[:2]
    
    src_points = np.float32([[0, 0], [width, 0], [width, height], [0, height]])

    p1 = [random.randint(0, int(width*max_distortion)), random.randint(0, int(height*max_distortion))]
    p2 = [random.randint(int(width*(1-max_distortion)), width), random.randint(0, int(height*max_distortion))]
    p3 = [random.randint(int(width*(1-max_distortion)), width), random.randint(int(height*(1-max_distortion)), height)]
    p4 = [random.randint(0, int(width*max_distortion)), random.randint(int(height*(1-max_distortion)), height)]
    
    dst_points = np.float32([p1, p2, p3, p4])
    
    max_width_distortion = abs(dst_points[1][0] - dst_points[0][0]) / width - 1
    max_height_distortion = abs(dst_points[3][1] - dst_points[0][1]) / height - 1
    
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    warped_image = cv2.warpPerspective(image, perspective_matrix, (width, height))

    corrected_box = []
    for bbox in boxes:
        bbox_points = np.float32([[[bbox[1], bbox[2]]], [[bbox[3], bbox[2]]], [[bbox[3], bbox[4]]], [[bbox[1], bbox[4]]]])
        warped_points = cv2.perspectiveTransform(bbox_points, perspective_matrix)

        warped_bbox = np.squeeze(warped_points).tolist()
        warped_bbox = [int(x) for point in warped_bbox for x in point] 
        corrected_box.append(tuple([bbox[0], warped_bbox[0], min(warped_bbox[1], warped_bbox[3]),
                                    warped_bbox[4], max(warped_bbox[5], warped_bbox[7])]))
    
    return warped_image, corrected_box

def unwarp(path):
    img = cv2.imread(path)
    GrayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    BlurredFrame = cv2.GaussianBlur(GrayImg, (5, 5), 1)
    CannyFrame = cv2.Canny(BlurredFrame, 190, 190)
    contours, _ = cv2.findContours(CannyFrame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    ContourFrame = img.copy()
    ContourFrame = cv2.drawContours(ContourFrame, contours, -1, (255, 0, 255), 4)
    CornerFrame = img.copy()
    maxArea = 0
    w = 512
    h = int(w*0.6785)
    biggest = []
    for i in contours :
        area = cv2.contourArea(i)
        if area > 2000 :
            peri = cv2.arcLength(i, True)
            edges = cv2.approxPolyDP(i, 0.02*peri, True)
            if area > maxArea and len(edges) == 4 :
                biggest = edges
                maxArea = area
    if len(biggest) != 0 :
        CornerFrame = cv2.drawContours(CornerFrame, biggest, -1, (255, 0, 255), 25)
        biggest = biggest.reshape((4,2))
        biggestNew = np.zeros((4,1,2), dtype=np.int32)
        add = biggest.sum(1)
        biggestNew[0] = biggest[np.argmin(add)]
        biggestNew[3] = biggest[np.argmax(add)]
        diff = np.diff(biggest, axis=1)
        biggestNew[1] = biggest[np.argmin(diff)]
        biggestNew[2] = biggest[np.argmax(diff)]
        pts1 = np.float32(biggestNew)
        pts2 = np.float32([[0,0],[w,0],[0,h],[w,h]])
        matrix = cv2.getPerspectiveTransform(pts1, pts2)
        warped = cv2.warpPerspective(img, matrix, (w,h))
        
    cv2.imwrite("scan.jpg", warped)
    display(ImageDisplay('scan.jpg'))

In [ ]:
def generate_random_data_korea():
    A_Z = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    fname = ''.join(random.choices(string.ascii_letters, k=random.randint(3, 12))).upper()
    lname = ''.join(random.choices(string.ascii_letters, k=random.randint(3, 12))).upper()
    name = f"{fname} {lname}"
    name1 = fake.name()
    name2 = fake.name()
    address = fake.address()
    #test line: 
    
    #passport_number = ''.join(random.choices(A_Z, k=1))+''.join(random.choices('0123456789', k=3))+''.join(random.choices(A_Z, k=1))+''.join(random.choices('0123456789', k=4))
    
    #months = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
    #random_month = random.choice(months)
    random_month1 = random.randint(1, 12)
    random_month2 = random.randint(1, 12)
    random_month3 = random.randint(1, 12)
    random_year_birth = random.randint(1950, 2024)
    random_year_issue = random_year_birth + random.randint(18,50)
    random_year_exp = random_year_issue + random.randint(5, 30)
    random_day = random.randint(1, 30)
    birthdate = f"{random_day}/{random_month1}/{random_year_birth}"
    issue_date = f"{random_day}/{random_month3}/{random_year_issue}"
    expiry_date = f"{random_day}/{random_month2}/{random_year_exp}"
    
    dlnum = ''.join(random.choices('0123456789', k=12))
    Class = ''.join(random.choices('ABCD', k= 1))
    #age = str(random.randint(18, 80))
    gender = random.choice(["M", "F"])
    # photo_path = "placeholder_photo.jpg"

    return name1, name2, dlnum, address, birthdate, expiry_date, issue_date, gender, Class#changed passport to dlnum here
    
def create_dummy_passport(template_path, output_path):
    # Load template image
    template = Image.open(template_path)
    
    draw = ImageDraw.Draw(template)
    font_path = "Lato-Regular.ttf"
    font = ImageFont.truetype(font_path, size=30)
    font_big = ImageFont.truetype(font_path, size=40)
    #font_kr = ImageFont.truetype(font_path_kr, size=30)

    name, name2, dlnum, address, birthdate, expiry_date, issue_date, gender, Class = generate_random_data_korea() #changed passport to dl num


    # Define text positions
    text_positions = {
        #changing all positions 
        "First_Name": ((416,304), name),
        "Last_Name": ((416, 337), name2), #father name (in india)
        #"Surname": ((430, 150), lname),
        "DLNumber": ((482, 221), dlnum),
        "Address": ((416, 377), address),
        "DOB": ((913, 227), birthdate),
        "ExpiryDate": ((913, 263), expiry_date),
        "IssueDate": ((466, 647), issue_date),
        "Sex": ((474, 565), gender),
        "Class": ((507, 505), Class),
        #"CountryCode": ((540, 150), "KOR"),
        #"Nationality": ((430, 485), "REPUBLIC OF KOREA")
    }
    boxes = []
    for key, position in text_positions.items():
      if(key == "Num"):
          #print("/n", key," next ", position[0][0], "-", position[0][1], "-", position[1])
          draw.text((position[0][0], position[0][1]-5), position[1], fill="black", font=font_big)
          bbox = draw.textbbox((position[0][0], position[0][1]-5), position[1], font=font_big)
          #print("/n", bbox)
          boxes.append(tuple([key])+bbox)
          #print("/n", boxes)
      else:       
         #print("/n", key," next ", position[0][0], "-", position[0][1], "-", position[1])
         draw.text((position[0][0], position[0][1]-8), position[1], fill="black", font=font)
         bbox = draw.textbbox((position[0][0], position[0][1]-5), position[1], font=font)
         #print("/n", bbox)
         boxes.append(tuple([key])+bbox)
         #print("/n", boxes)
        
    # photo = Image.open(photo_path)
    # photo = photo.resize((120, 150))
    # template.paste(photo, (300, 100))
    
    image_path = output_path.replace("***", "images")+".jpg"
    json_path = output_path.replace("***", "json")+".json"
    
    # Create directories if they do not exist
    os.makedirs(os.path.dirname(image_path), exist_ok=True)
    os.makedirs(os.path.dirname(json_path), exist_ok=True)
    
    template.save(image_path)
    image, corrected_box = random_size(image_path, boxes)
    if(random.randint(0,1)==1):
        image, corrected_box = perspective_warp(image, corrected_box, max_distortion=0.1)
    # for (_, xmin, ymin, xmax, ymax) in corrected_box:
    #     image = cv2.rectangle(image, (xmin, ymin), (xmax, ymax),  color=(255,0,0), thickness=1)

    label_to_json(corrected_box, image_path.split('images/')[1], json_path, image.shape[1], image.shape[0])
    cv2.imwrite(output_path.replace("***", "images")+".jpg", image)
    return image_path


def generate_images(num):
    for i in range(num):
        i = "synthetic_data/train/carolina/***/caro"+str(i) #changed k to g
        template_path = "north_carolina - Copy.jpg" 
        output_path = i
        file = create_dummy_passport(template_path, output_path)
        display(ImageDisplay(filename=file))
        
generate_images(120)
# unwarp("synthetic_data/korea/images/0.jpg")

In [ ]:
import shutil

def zip_directory(directory, zip_filename):
    # Compress the directory into a zip file
    shutil.make_archive(zip_filename, 'zip', directory)

# Specify the directory to compress and the desired zip file name
directory_to_compress = "synthetic_data/train/carolina/images"
zip_file_name = "carolina_images"

# Call the function to compress the directory
zip_directory(directory_to_compress, zip_file_name)

In [ ]:
# Specify the directory to compress and the desired zip file name
directory_to_compress = "synthetic_data/train/carolina/json"
zip_file_name = "carolina_json"

# Call the function to compress the directory
zip_directory(directory_to_compress, zip_file_name)